In [ ]:
from pathlib import Path

import matplotlib.patches as patches
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib.path import Path as mplPath

sns.set_theme(style="ticks", palette="pastel")
sns.set_style("white")
# sns.set_context("poster")
sns.set_context("talk")

In [ ]:
data_dir = Path(".")
replicate_name = "trial"

limits = {
    "Wheel Radius": (0.5, 1.5),
    "Chassis Length": (1, 4),
    "Suspension Frequency": (1, 8),
    "Suspension Damping": (0.3, 0.9),
    "Sensor Limit": (1, 15),
    "Speed Max": (0, 10),
    "Speed Slope": (0, 10),
    "Speed Intercept": (-20, 20),
}

objectives = {
    "Final Distance": (0, 17),
    "Final Speed": (0, 10),
    "Hit Wall": (0, 1),
    "Wheel Radius": (0.5, 1.5),
    "Index At Rest": (0, 2008),
}

In [ ]:
files = data_dir.glob("*-generations.csv")

partial_dfs = []

for f in files:
    trial = f.stem.split(replicate_name)[1].split("-")[0]
    df_partial = pd.read_csv(f)
    df_partial["Trial"] = trial

    partial_dfs.append(df_partial)

df = pd.concat(partial_dfs, ignore_index=True)
df

In [ ]:
sns_plot = sns.lineplot(x="Generation", y="Best Objective", data=df)
sns.lineplot(x="Generation", y="Average Objective", data=df)
plt.legend(["Best", "Best-Conf", "Average", "Average-Conf"])
# sns_plot.figure.savefig("generations.png")  # type: ignore
sns.despine()

In [ ]:
files = data_dir.glob("*-population.csv")

partial_dfs = []

for f in files:
    trial = f.stem.split(replicate_name)[1].split("-")[0]
    df_partial = pd.read_csv(f)
    df_partial["Trial"] = trial

    partial_dfs.append(df_partial)

df = pd.concat(partial_dfs, ignore_index=True)
df.columns = [n.replace("_", " ").replace("-", " ").title() for n in df.columns]
df

In [ ]:
n = len(limits)

fig, axes = plt.subplots(1, n, figsize=(20, 5))

for ax, name in zip(axes, limits.keys()):
    lo, hi = limits[name]
    sns.stripplot(y=name, data=df, ax=ax)
    ax.set_ylabel("")
    ax.set_xlabel("".join(word[:2] for word in name.split(" ")))
    ax.set_ylim(lo, hi)
    ax.set_yticks([lo, (lo + hi) / 2, hi])

sns.despine()

In [ ]:
from pandas.plotting import parallel_coordinates

genome_names = [n + " Genome" for n in limits.keys()]
data = [list(df[n].values) for n in genome_names]
data = np.array(data)

df_genomes = df[genome_names + ["Trial"]]
parallel_coordinates(df_genomes, "Trial")

In [ ]:
n = len(objectives)

fig, axes = plt.subplots(1, n, figsize=(20, 5))

# df["Index At Rest"] = df["Index At Rest"].astype(int)

for ax, name in zip(axes, objectives.keys()):
    lo, hi = objectives[name]
    sns.stripplot(y=name, data=df, ax=ax)
    ax.set_ylabel("")
    ax.set_xlabel("".join(word[:2] for word in name.split(" ")))
    ax.set_ylim(lo, hi)
    ax.set_yticks([lo, (lo + hi) / 2, hi])

sns.despine()

In [ ]:
# parallelCoordinatesPlot(
#     title="title",
#     N=len(df_genomes),
#     data=data,
#     category=df_genomes["cat"].astype("int"),
#     ynames=["".join(w[:2] for w in n.split(" ")[:2]) for n in genome_names],
# )

In [ ]:
# def parallelCoordinatesPlot(
#     title, N, data, category, ynames, colors=None, category_names=None
# ):
#     """
#     A legend is added, if category_names is not None.

#     :param title: The title of the plot.
#     :param N: Number of data sets (i.e., lines).
#     :param data: A list containing one array per parallel axis, each containing N data points.
#     :param category: An array containing the category of each data set.
#     :param category_names: Labels of the categories. Must have the same length as set(category).
#     :param ynames: The labels of the parallel axes.
#     :param colors: A colormap to use.
#     :return:
#     """

#     fig, host = plt.subplots()

#     # organize the data
#     ys = np.dstack(data)[0]
#     ymins = ys.min(axis=0)
#     ymaxs = ys.max(axis=0)
#     dys = ymaxs - ymins
#     ymins -= dys * 0.05  # add 5% padding below and above
#     ymaxs += dys * 0.05
#     dys = ymaxs - ymins

#     # transform all data to be compatible with the main axis
#     zs = np.zeros_like(ys)
#     zs[:, 0] = ys[:, 0]
#     zs[:, 1:] = (ys[:, 1:] - ymins[1:]) / dys[1:] * dys[0] + ymins[0]

#     axes = [host] + [host.twinx() for i in range(ys.shape[1] - 1)]
#     for i, ax in enumerate(axes):
#         # ax.set_ylim(ymins[i], ymaxs[i])
#         ax.set_ylim(0, 1)
#         ax.spines["top"].set_visible(False)
#         ax.spines["bottom"].set_visible(False)
#         if ax != host:
#             ax.spines["left"].set_visible(False)
#             ax.yaxis.set_ticks_position("right")
#             ax.spines["right"].set_position(("axes", i / (ys.shape[1] - 1)))

#     host.set_xlim(0, ys.shape[1] - 1)
#     host.set_xticks(range(ys.shape[1]))
#     host.set_xticklabels(ynames, fontsize=14)
#     host.tick_params(axis="x", which="major", pad=7)
#     host.spines["right"].set_visible(False)
#     host.xaxis.tick_top()
#     host.set_title(title, fontsize=18)

#     if colors is None:
#         colors = plt.cm.tab10.colors
#     if category_names is not None:
#         legend_handles = [None for _ in category_names]
#     else:
#         legend_handles = [None for _ in set(category)]
#     for j in range(N):
#         # to just draw straight lines between the axes:
#         # host.plot(range(ys.shape[1]), zs[j,:], c=colors[(category[j] - 1) % len(colors) ])

#         # create bezier curves
#         # for each axis, there will a control vertex at the point itself, one at 1/3rd towards the previous and one
#         #   at one third towards the next axis; the first and last axis have one less control vertex
#         # x-coordinate of the control vertices: at each integer (for the axes) and two inbetween
#         # y-coordinate: repeat every point three times, except the first and last only twice
#         verts = list(
#             zip(
#                 [
#                     x
#                     for x in np.linspace(0, len(ys) - 1, len(ys) * 3 - 2, endpoint=True)
#                 ],
#                 np.repeat(zs[j, :], 3)[1:-1],
#             )
#         )
#         # for x,y in verts: host.plot(x, y, 'go') # to show the control points of the beziers
#         codes = [mplPath.MOVETO] + [mplPath.CURVE4 for _ in range(len(verts) - 1)]
#         path = mplPath(verts, codes)
#         patch = patches.PathPatch(
#             path, facecolor="none", lw=1, edgecolor=colors[category[j] - 1]
#         )
#         legend_handles[category[j] - 1] = patch
#         host.add_patch(patch)

#         if category_names is not None:
#             host.legend(
#                 legend_handles,
#                 category_names,
#                 loc="lower center",
#                 bbox_to_anchor=(0.5, -0.18),
#                 ncol=len(category_names),
#                 fancybox=True,
#                 shadow=True,
#             )

#     plt.tight_layout()
#     plt.show()